In [1]:
#importar bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import time
import random
import re

In [2]:
# Lista de URL para recolha de dados

#https://portuguesecharts.com/archive.asp?sparte=a&woche=1&jahr=2004&todo=show

static = 'https://portuguesecharts.com/archive.asp?sparte=a&woche='
static2 = '&jahr='
static3 = '&todo=show'

urllist = [static+str(week)+static2+str(year)+static3 for year in range (2004,2020) for week in range(1,53)]

## Ciclo de recolha de dados

In [3]:
maindata = pd.DataFrame()

#urllist_test = urllist[23:25]

for i in tqdm(urllist):
    headers = requests.utils.default_headers()
    headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
    URL = i
    year = str(re.findall(r'(\d+)(?!.*\d)', URL))
    cal_week = str(re.findall(r'^\D*(\d+)', URL))
    page = requests.get(URL)
    
    soup = BeautifulSoup(page.content, "lxml")
    
    chart_table = [name.text for name in soup.find_all("td", attrs = {'class':'text'})] 
    chart_table  = chart_table[1:] #remove first useless thing
    
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]
    
    chart = list(chunks(chart_table, 7))
    #print(chart)
    chart_data = pd.DataFrame(chart)
    chart_data.reset_index(inplace=True)
    
    chart_data["calendar_week"] = cal_week
    chart_data["year"] = year
    maindata = pd.concat([maindata, chart_data])
    time.sleep(random.randint(1,2))
    
print("Já fomos lá buscar tudo")


Finished!!!!!


In [4]:
maindata

,index,0,1,2,3,4,5,6,calendar_week,year
0,0,1,(1),,Rui Veloso,O concerto acústico,Virgin,8,['1'],['2004']
1,1,2,(3),,Robbie Williams,Live Summer 2003,Chrysalis,14,['1'],['2004']
2,2,3,(2),,Tribalistas,Tribalistas,Emi,42,['1'],['2004']
3,3,4,(4),,Linkin Park,Live In Texas,Wb,7,['1'],['2004']
4,4,5,(6),,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,['1'],['2004']
...,...,...,...,...,...,...,...,...,...,...
45,45,46,(++),,Soundtrack / António Variações,Variações,Sony Music,13,['52'],['2019']
46,46,47,(44),,Cigarettes After Sex,Cry,Altafonte,8,['52'],['2019']
47,47,48,(++),,Melim,Melim,Universal,4,['52'],['2019']
48,48,49,(++),,Taylor Swift,Lover,Universal,12,['52'],['2019']


## Limpar os dados para exportação

In [5]:
maindata.dtypes

index            object
0                object
1                object
2                object
3                object
4                object
5                object
6                object
calendar_week    object
year             object
dtype: object

In [6]:
maindata.shape

(28970, 10)

In [7]:
maindata.rename(columns = {0: "rank", 1:"previous_week", 2: "empty", 3:"artist", 4: "album", 5:"label", 6:"weeks_on_chart"}, inplace=True)
maindata

,index,rank,previous_week,empty,artist,album,label,weeks_on_chart,calendar_week,year
0,0,1,(1),,Rui Veloso,O concerto acústico,Virgin,8,['1'],['2004']
1,1,2,(3),,Robbie Williams,Live Summer 2003,Chrysalis,14,['1'],['2004']
2,2,3,(2),,Tribalistas,Tribalistas,Emi,42,['1'],['2004']
3,3,4,(4),,Linkin Park,Live In Texas,Wb,7,['1'],['2004']
4,4,5,(6),,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,['1'],['2004']
...,...,...,...,...,...,...,...,...,...,...
45,45,46,(++),,Soundtrack / António Variações,Variações,Sony Music,13,['52'],['2019']
46,46,47,(44),,Cigarettes After Sex,Cry,Altafonte,8,['52'],['2019']
47,47,48,(++),,Melim,Melim,Universal,4,['52'],['2019']
48,48,49,(++),,Taylor Swift,Lover,Universal,12,['52'],['2019']


In [8]:
maindata.drop(['empty', "index"], axis=1, inplace = True)

In [9]:
maindata['previous_week'] = maindata['previous_week'].str.strip("()")
maindata

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,['1'],['2004']
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,['1'],['2004']
2,3,2,Tribalistas,Tribalistas,Emi,42,['1'],['2004']
3,4,4,Linkin Park,Live In Texas,Wb,7,['1'],['2004']
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,['1'],['2004']
...,...,...,...,...,...,...,...,...
45,46,++,Soundtrack / António Variações,Variações,Sony Music,13,['52'],['2019']
46,47,44,Cigarettes After Sex,Cry,Altafonte,8,['52'],['2019']
47,48,++,Melim,Melim,Universal,4,['52'],['2019']
48,49,++,Taylor Swift,Lover,Universal,12,['52'],['2019']


In [12]:
maindata['calendar_week'] = maindata['calendar_week'].str.strip("['']")
maindata["year"] = maindata["year"].str.strip("['']")
maindata

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,1,2004
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,1,2004
2,3,2,Tribalistas,Tribalistas,Emi,42,1,2004
3,4,4,Linkin Park,Live In Texas,Wb,7,1,2004
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,1,2004
...,...,...,...,...,...,...,...,...
45,46,++,Soundtrack / António Variações,Variações,Sony Music,13,52,2019
46,47,44,Cigarettes After Sex,Cry,Altafonte,8,52,2019
47,48,++,Melim,Melim,Universal,4,52,2019
48,49,++,Taylor Swift,Lover,Universal,12,52,2019


In [13]:
maindata.to_csv("charts30-50.csv")

In [35]:
#maindata.astype({'rank': 'int64', 'weeks_on_chart': 'int64', "calendar_week" : 'int64', "year": 'int64'}).dtypes
charts = pd.read_csv("charts30-50.csv")

In [38]:
charts = charts[charts["rank"] < 31]
charts.to_csv("charts.csv")